In [1]:
import json
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Normalizer

In [2]:
# Constants

# File paths
DATA_FP = "./data/" # Data file path
TWEETS_FP = DATA_FP + "tweets.json"
TRAIN_DEV_FP = DATA_FP + "labels-train+dev.tsv"
TEST_FP = DATA_FP + "labels-test.tsv"

# Column names
COL_ID = 'ID'
COL_TWEET = 'Tweet'
COL_LABEL = 'Label'

---
# Data

## Tweets

In [3]:
# process the first file (Tweets)
tweets = []
with open(TWEETS_FP, 'r') as tweets_fh:  # Tweets file handle
    for line in tweets_fh:   # put each line in a list of lines
        j_content = json.loads(line)
        tweets.append(j_content)

tweets = pd.DataFrame(tweets, columns=[COL_ID, COL_TWEET])  # make a dataframe out of it

In [4]:
tweets # looks good!

,ID,Tweet
0,483885347374243841,اللهم أفرح قلبي وقلب من أحب وأغسل أحزاننا وهمو...
1,484023414781263872,إضغط على منطقتك يتبين لك كم يتبقى من الوقت عن ...
2,484026168300273664,اللَّهٌمَّ صَلِّ وَسَلِّمْ عَلىٰ نَبِيِّنَآ مُ...
3,483819942878650369,@Dinaa_ElAraby اها يا بيبي والله اتهرست علي تو...
4,483793769079123971,• افضل كتاب قرأته هو : أمي (ابراهام لنكولن)\n🌹...
5,483934868070350849,@hudc7721 انتظري اجل \nخيره لك يارب 😘
6,483863369972473856,(وإن تجهر بالقول فإنه يعلم السر وأخفى) [طه:7]\...
7,483871567311413248,ﺧﻟك ﻋزﯾز آﻟﻧﻓس ﻟۈ ﮪﻣۈﻣك ﺟﺑآللاﭠﺷﺷﮐي ﻟﻟﻧآﺳس ﻣن ...
8,483931429902884864,عشان الجنّة أجمل ؟ الله يبعدنا عن كل ذنب مايخ...
9,483773756897124352,توجيه كيفية تثبيت البرامج الثابتة ROM التحميل ...


## Labels

In [5]:
# deal with both label documents

train_dev_labels = pd.read_csv(TRAIN_DEV_FP, sep='\t', header=None, names=[COL_LABEL, COL_ID])
test_labels = pd.read_csv(TEST_FP, sep='\t', header=None, names=[COL_LABEL, COL_ID])

In [6]:
train_dev_labels # looks about right

,Label,ID
0,ar,483762194908479488
1,ar,483762916097654784
2,ar,483764828784582656
3,ar,483765526683209728
4,ar,483768342315282432
5,ar,483770765985083392
6,ar,483770900127285248
7,ar,483770997892345857
8,ar,483773690769702912
9,ar,483773756897124352


In [7]:
tweets[COL_ID]=tweets[COL_ID].astype(int) # to allow for merge, need the same type

train_dev_data = pd.merge(tweets, train_dev_labels, on=COL_ID) # merge by ID
test_data = pd.merge(tweets, test_labels, on=COL_ID) # merge by ID

In [8]:
train_dev_data.head()

,ID,Tweet,Label
0,483885347374243841,اللهم أفرح قلبي وقلب من أحب وأغسل أحزاننا وهمو...,ar
1,484023414781263872,إضغط على منطقتك يتبين لك كم يتبقى من الوقت عن ...,ar
2,484026168300273664,اللَّهٌمَّ صَلِّ وَسَلِّمْ عَلىٰ نَبِيِّنَآ مُ...,ar
3,483819942878650369,@Dinaa_ElAraby اها يا بيبي والله اتهرست علي تو...,ar
4,483793769079123971,• افضل كتاب قرأته هو : أمي (ابراهام لنكولن)\n🌹...,ar


In [9]:
test_data.tail() 

,ID,Tweet,Label
13447,491992347278725120,『 』 免費的女性向戀愛遊戲★想和哥哥假結婚看看嗎？ https://t.co/YOUkKU...,zh-TW
13448,494650331528433666,#204.7.25 花蓮區漁會 http://t.co/yOukLkXwCr,zh-TW
13449,485752157937741824,@makzihau 当然是史地啦,zh-CN
13450,484586108282736641,@Official_SABC1 Moloooo nakuwe!!!,zu
13451,484672019091300352,精神統一精神統一精神統一精神統一精神統一精神統一精神統一精神統一精神統一精神統一精神統一精神...,zh-TW


In [10]:
def drop_n_shuffle(data):
    data_no_na = data.dropna().copy()
    return data_no_na.sample(frac=1)

train_dev_set = drop_n_shuffle(train_dev_data).reset_index(drop = True)
test_set = drop_n_shuffle(test_data)

In [11]:
print(train_dev_set.head()) #some checks
print(type(train_dev_set))
print(test_set.head())
print(type(test_set))

                   ID                                              Tweet Label
0  490917308399763457   "Hold on I'm going to brush a horse." -Willow 😂😂    en
1  493347986013822976  @atsushi022518 \nあっちゃん、てんねんじゃないよ。天才だよ💫おさとうはまちが...    ja
2  484938267268091904         Упругое тело… (18+) http://t.co/xpXN8rDqED    ru
3  490853889978007552                               @chiyari_ Thanks !!!    en
4  490161646678327296                                 ישראל היא הבית שלי    he
<class 'pandas.core.frame.DataFrame'>
                       ID                                              Tweet  \
11820  484392947555643392  @mk62442 @P0406L เดี๋ยวให้โวลเดอร์มอต์พาตัวมา ^.^   
11022  486685688243699713                               Segunda tem aula -.-   
11843  487509040751054850                       ลิมิตหรอ กี้ #SbsPopAsiaGOT7   
11142  488795219643858945        chego quem desconcentra o duca #ODucaChegou   
3044   489696421713833984  @Ferenc2017 Grrrrrrr!!!!! They have some splai...   

      L

In [12]:
# drop the ID columns, not needed anymore

train_dev = train_dev_set.drop(COL_ID, axis=1)
test = test_set.drop(COL_ID, axis=1)

In [14]:
train_dev.head()

,Tweet,Label
0,"""Hold on I'm going to brush a horse."" -Willow 😂😂",en
1,@atsushi022518 \nあっちゃん、てんねんじゃないよ。天才だよ💫おさとうはまちが...,ja
2,Упругое тело… (18+) http://t.co/xpXN8rDqED,ru
3,@chiyari_ Thanks !!!,en
4,ישראל היא הבית שלי,he


In [15]:
test.head()

,Tweet,Label
11820,@mk62442 @P0406L เดี๋ยวให้โวลเดอร์มอต์พาตัวมา ^.^,th
11022,Segunda tem aula -.-,pt
11843,ลิมิตหรอ กี้ #SbsPopAsiaGOT7,th
11142,chego quem desconcentra o duca #ODucaChegou,pt
3044,@Ferenc2017 Grrrrrrr!!!!! They have some splai...,en


In [16]:
train_dev.describe() # descrption of data

,Tweet,Label
count,53469,53469
unique,53385,77
top,:(,en
freq,8,18764


In [17]:
train_dev.groupby(COL_LABEL).size() # more description of the data. See that there are lots of Arabian and English tweets, also quite a few in Spanish and Portuguese

Label
ar          2295
ar             1
ar_LATN       12
az             1
bg             2
bn             8
bs             4
ca            22
cs             4
cy             1
da             7
de           171
dv             1
el            39
en         18764
es          5978
et             2
fa            18
fi            15
fr           954
gl             3
ha             1
he            27
hi            16
hi-Latn       15
hr             5
ht             2
hu            15
hy             2
id          3038
           ...  
nl           182
no            11
pl            93
ps             1
ps_LATN        1
pt          2888
ro            12
ru           978
si             1
sl             2
sq             9
sr            22
su            10
sv            54
sw             6
ta             9
ta_LATN        1
th           465
tl           320
tn             1
tr           669
uk            16
und         4835
ur             7
ur_LATN       12
vi            16
wo             1
xh      

In [18]:
X_train = train_dev.Tweet # split the data in Series
y_train = train_dev.Label
X_test = test.Tweet
y_test = test.Label

In [19]:
print(type(X_train))
print(type(y_train))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [20]:
from sklearn.preprocessing import LabelEncoder
#encode the labels. First step means simple encoding, the second makes a series out of the array that was outputted and
    #the third step means we output strings again (strings are apparently needed as a format)

label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_train_series = pd.Series(y_train_encoded)
y_train_str = y_train_series.apply(str)

y_test_encoded = label_encoder.fit_transform(y_test)
y_test_series = pd.Series(y_test_encoded)
y_test_str = y_test_series.apply(str)

# Part 1 - Language classification with linear classification

In [22]:
#pipeline1 = Pipeline([('tfidf', TfidfVectorizer()), ('clf0', MultinomialNB())]) #first test with a pipeline

In [23]:
#pipeline1.fit(X_train, y_train_str) #using y_train_str

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...ue,
        vocabulary=None)), ('clf0', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [24]:
#pipeline1.predict(X_test)

array(['28', '28', '31', ..., '13', '13', '13'], dtype='<U2')

In [25]:
#y_test_str #doesn't look too right, but it is a very simple model

0        21
1        21
2        23
3        10
4        21
5        23
6        10
7        34
8        10
9        21
10       11
11       11
12        0
13        0
14       50
15        0
16       10
17       23
18       23
19       10
20       23
21       10
22       10
23       10
24       10
25       23
26       11
27       50
28       11
29       10
         ..
13422    23
13423    10
13424    10
13425     0
13426    41
13427    10
13428     0
13429    39
13430    10
13431    23
13432    23
13433    10
13434    23
13435    10
13436    10
13437    10
13438    52
13439    10
13440    10
13441    23
13442    10
13443    10
13444    10
13445    10
13446    41
13447    10
13448    11
13449     0
13450     3
13451    23
Length: 13452, dtype: object

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer()

In [22]:
#first serious pipeline with ngrams and tfidf --> just for test!
pipeline_NB01 = Pipeline([
    ('ngram', CountVectorizer(ngram_range=(1, 4), analyzer='word')),
    ('tfidf', TfidfTransformer()),
    ('clf01', MultinomialNB())
])

In [23]:
#param_grid is fitting the pipeline_NB01
param_grid01 = {'clf01__alpha': [0.2, 0.6, 0.8, 1.0],
                 'clf01__fit_prior': [True, False]}  #'ngram__ngram_range': [(1, 1), (1, 2), (1, 4)]


In [24]:
#This model here seems to work
gs_NB01= GridSearchCV(pipeline_NB01, param_grid01, cv=2, n_jobs=2, verbose=1)
gs_NB01.fit(X_train, y_train)

Fitting 2 folds for each of 8 candidates, totalling 16 fits


/Users/debora/Envs/nlppython/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=2.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  16 out of  16 | elapsed:  1.3min finished


GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('ngram', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 4), preprocessor=None, stop_words=None,
        stri...ear_tf=False, use_idf=True)), ('clf01', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params=None, iid='warn', n_jobs=2,
       param_grid={'clf01__alpha': [0.2, 0.6, 0.8, 1.0], 'clf01__fit_prior': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [25]:
y_NB01 = gs_NB01.predict(X_test)
y_NB01

array(['th', 'pt', 'th', ..., 'pt', 'es', 'tl'], dtype='<U7')

In [26]:
y_test #results are quite terrible though 
# they are actually better than with the avg word length extractor included in the features :D wtf o_o

11820         th
11022         pt
11843         th
11142         pt
3044          en
3753          en
12143         tr
9873          ja
7079          fr
3522          en
2427          en
8955          ja
11056         pt
2266          en
10209         ja
6013          es
2235          en
4007          en
7556          id
4080          en
10281         ja
9641          ja
547      ar_LATN
9551          ja
5369          en
10189         ja
550           bg
7831          id
1347          en
10686         ko
          ...   
8269          ja
11124         pt
5627          es
1532          en
2264          en
7119          fr
6286          es
1817          en
1497          en
2817          en
10927         pt
6646          es
9908          ja
4252          en
8662          ja
9708          ja
2150          en
6090          es
7868          id
11810         sv
9268          ja
9170          ja
11331         pt
6446          es
1287          en
2569          en
7286          id
13226        u

In [27]:
accuracy_score(y_test, y_NB01) 

0.7354296758846268

But here we really need to find the best_model out of this data i got. I don't remember how to see it in the matrix, see Tutorial!

In [28]:
#Average word length extractor, inspired  by https://michelleful.github.io/code-blog/2015/06/20/pipelines/)
class AverageWordLengthExtractor(BaseEstimator, TransformerMixin):
    """Takes in dataframe, extracts tweet column, outputs average word length"""

    def __init__(self):
        pass

    def average_word_length(self, tweet):
        """Helper code to compute average word length of a tweet"""
        return np.mean([len(word) for word in tweet.split()])

    def transform(self, df, y=None):
        """The workhorse of this feature extractor"""
        # the result of the transform needs to be a 2d array a.k.a. dataframe
        # https://stackoverflow.com/a/50713209
        result = df.apply(self.average_word_length).to_frame()
        print(result)
        return result

    def fit(self, df, y=None):
        """Returns `self` unless something different happens in train and test"""
        return self

In [29]:
# read this blog post on how to construct feature unions :) 
# http://zacstewart.com/2014/08/05/pipelines-of-featureunions-of-pipelines.html

# the problem was that our previous pipeline went sorta like this:
# create n-grams from text (CountVectorizer) -> tfidf from ngrams (TfidfTransformer) -> average length from ngrams (AvgWLExtractor) ummm :) that wouldn't work... we need to compute the average word length from the original data (tweets / strings).
# that's why we have to do these two "pipelines" separately => now we just compute the avg from the strings like so...

pipeline_NB1 = Pipeline([
    ('features', FeatureUnion([
        # first feature
        ('ngram_tfidf', Pipeline([
            ('ngram', CountVectorizer(ngram_range=(1, 4), analyzer='word')),
            ('tfidf', TfidfTransformer())
        ])),
        # second feature
        ('ave_scaled', Pipeline([
            ('ave', AverageWordLengthExtractor()),
            ('scale', Normalizer())
        ]))
    ])),
    ('nb_clf', MultinomialNB()) # classifier
])

In [30]:
#-*- coding: utf-8 -*-
y_train.describe()

count     53469
unique       77
top          en
freq      18764
Name: Label, dtype: object

In [31]:
param_grid1 = {'nb_clf__alpha': [0.2, 0.6, 0.8, 1.0],
                 'nb_clf__fit_prior': [True, False]}  #'ngram__ngram_range': [(1, 1), (1, 2), (1, 4)]


In [32]:
gs_NB1= GridSearchCV(pipeline_NB1, param_grid1, cv=10, n_jobs=2, verbose=1)
gs_NB1.fit(X_train, y_train)

Fitting 10 folds for each of 8 candidates, totalling 80 fits


/Users/debora/Envs/nlppython/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  7.3min
[Parallel(n_jobs=2)]: Done  80 out of  80 | elapsed: 12.6min finished


            Tweet
0        3.900000
1       25.000000
2        9.750000
3        6.000000
4        3.750000
5        5.142857
6        4.190476
7        4.523810
8       10.800000
9        6.200000
10       5.000000
11      13.500000
12      16.500000
13       9.500000
14       3.000000
15       3.250000
16       7.333333
17       4.833333
18       8.133333
19       5.000000
20      37.000000
21       5.000000
22      12.000000
23       4.421053
24       6.388889
25      15.666667
26      12.000000
27       6.333333
28       4.777778
29      11.500000
...           ...
53439    7.600000
53440    4.684211
53441   12.000000
53442    8.333333
53443    5.000000
53444    4.000000
53445    9.500000
53446    9.300000
53447   18.285714
53448    6.500000
53449  140.000000
53450   10.500000
53451   12.000000
53452    4.769231
53453    4.166667
53454    4.153846
53455    4.400000
53456    5.619048
53457   10.000000
53458   22.500000
53459    5.809524
53460    5.000000
53461    4.950000
53462    7

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('ngram_tfidf', Pipeline(memory=None,
     steps=[('ngram', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max... transformer_weights=None)), ('nb_clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params=None, iid='warn', n_jobs=2,
       param_grid={'nb_clf__alpha': [0.2, 0.6, 0.8, 1.0], 'nb_clf__fit_prior': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [33]:
y_NB1 = gs_NB1.predict(X_test)

           Tweet
11820  11.500000
11022   4.250000
11843   8.666667
11142   6.333333
3044    5.461538
3753    9.250000
12143   8.571429
9873   21.000000
7079    4.733333
3522    5.571429
2427    4.750000
8955   20.500000
11056   5.000000
2266    6.210526
10209  17.333333
6013    4.625000
2235    4.142857
4007    4.600000
7556    6.444444
4080    8.800000
10281  11.000000
9641   14.333333
547     7.400000
9551   24.666667
5369    4.956522
10189  41.000000
550     9.000000
7831    4.421053
1347    4.000000
10686   3.500000
...          ...
8269   10.000000
11124   4.625000
5627    6.066667
1532    4.272727
2264    6.250000
7119    6.142857
6286    3.727273
1817    9.875000
1497    4.153846
2817    6.833333
10927   3.500000
6646    7.666667
9908   13.500000
4252    7.562500
8662    7.500000
9708   14.000000
2150    7.500000
6090    4.352941
7868    7.125000
11810   4.200000
9268    5.000000
9170   20.000000
11331   5.000000
6446   13.000000
1287    7.642857
2569    5.500000
7286    5.4000

In [35]:
grid_result = pd.DataFrame.from_dict(gs_NB1.cv_results_)
grid_result.sort_values(by=["rank_test_score"])

/Users/debora/Envs/nlppython/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/debora/Envs/nlppython/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/debora/Envs/nlppython/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/debora

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_nb_clf__alpha,param_nb_clf__fit_prior,params,split0_test_score,split1_test_score,split2_test_score,...,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,split8_train_score,split9_train_score,mean_train_score,std_train_score
1,13.450784,1.215598,0.985117,0.315522,0.2,False,"{'nb_clf__alpha': 0.2, 'nb_clf__fit_prior': Fa...",0.683605,0.678771,0.683159,...,0.974581,0.974478,0.974211,0.974008,0.974218,0.973700,0.973579,0.973873,0.974189,0.000385
3,11.846274,1.214728,0.854151,0.102933,0.6,False,"{'nb_clf__alpha': 0.6, 'nb_clf__fit_prior': Fa...",0.631120,0.624022,0.623413,...,0.873735,0.874138,0.873673,0.873613,0.873499,0.873507,0.873214,0.872544,0.873699,0.000599
0,13.255900,0.982897,1.107839,0.231791,0.2,True,"{'nb_clf__alpha': 0.2, 'nb_clf__fit_prior': True}",0.632047,0.622160,0.624160,...,0.897408,0.897685,0.897882,0.897112,0.897681,0.897542,0.897453,0.896469,0.897525,0.000492
5,12.023044,0.746523,0.908931,0.099870,0.8,False,"{'nb_clf__alpha': 0.8, 'nb_clf__fit_prior': Fa...",0.610163,0.604097,0.606049,...,0.815954,0.814698,0.815694,0.815583,0.814809,0.814758,0.814909,0.814621,0.815145,0.000442
7,13.565887,1.911244,0.987051,0.152788,1,False,"{'nb_clf__alpha': 1.0, 'nb_clf__fit_prior': Fa...",0.591246,0.586592,0.584578,...,0.772348,0.772009,0.771950,0.771369,0.771056,0.771444,0.771332,0.770924,0.771562,0.000422
2,12.472430,0.848787,0.986116,0.202387,0.6,True,"{'nb_clf__alpha': 0.6, 'nb_clf__fit_prior': True}",0.537463,0.533333,0.530246,...,0.704778,0.704152,0.704017,0.704072,0.703287,0.703077,0.703307,0.703718,0.703841,0.000480
4,13.750000,2.288363,1.149593,0.388676,0.8,True,"{'nb_clf__alpha': 0.8, 'nb_clf__fit_prior': True}",0.509829,0.504842,0.502614,...,0.615676,0.614619,0.615885,0.615188,0.615282,0.615119,0.613804,0.614683,0.615108,0.000572
6,13.163292,1.994885,1.202061,0.620069,1,True,"{'nb_clf__alpha': 1.0, 'nb_clf__fit_prior': True}",0.489800,0.485661,0.484317,...,0.504978,0.503720,0.504167,0.503678,0.504508,0.504041,0.502181,0.504133,0.503966,0.000693


We can see that the best model is with an alpha of 0.2 and a fit_prior: False.

In [36]:
best_model = Pipeline([
    ('features', FeatureUnion([
        # first feature
        ('ngram_tfidf', Pipeline([
            ('ngram', CountVectorizer(ngram_range=(1, 4), analyzer='word')),
            ('tfidf', TfidfTransformer())
        ])),
        # second feature
        ('ave_scaled', Pipeline([
            ('ave', AverageWordLengthExtractor()),
            ('scale', Normalizer())
        ]))
    ])),
    ('nb_clf', MultinomialNB(alpha=0.2, fit_prior=False)) # classifier
])

best_model.fit(X_train, y_train)
best_model_prediction = best_model.predict(X_test)

            Tweet
0        3.900000
1       25.000000
2        9.750000
3        6.000000
4        3.750000
5        5.142857
6        4.190476
7        4.523810
8       10.800000
9        6.200000
10       5.000000
11      13.500000
12      16.500000
13       9.500000
14       3.000000
15       3.250000
16       7.333333
17       4.833333
18       8.133333
19       5.000000
20      37.000000
21       5.000000
22      12.000000
23       4.421053
24       6.388889
25      15.666667
26      12.000000
27       6.333333
28       4.777778
29      11.500000
...           ...
53439    7.600000
53440    4.684211
53441   12.000000
53442    8.333333
53443    5.000000
53444    4.000000
53445    9.500000
53446    9.300000
53447   18.285714
53448    6.500000
53449  140.000000
53450   10.500000
53451   12.000000
53452    4.769231
53453    4.166667
53454    4.153846
53455    4.400000
53456    5.619048
53457   10.000000
53458   22.500000
53459    5.809524
53460    5.000000
53461    4.950000
53462    7

In [37]:
accuracy_score(y_test, best_model_prediction)

0.6885964912280702

In [38]:
accuracy_score(y_test, y_NB1)
# not rly sure why the accuracy is lower than NB01, kinda confused bout this :D

0.6885964912280702

In [43]:
# Let's test without avg but with a nested FeatureUnion with 1 feature (pipeline)
# This should have the same accuracy as NB01, and it does... which implies that the pipeline/featureunion thing is correct. I'd say we have to look into hyperparams to fix the accuracy.
pipeline_NB2 = Pipeline([
    ('features', FeatureUnion([
        # first feature
        ('ngram_tfidf', Pipeline([
            ('ngram', CountVectorizer(ngram_range=(1, 4), analyzer='word')),
            ('tfidf', TfidfTransformer())
        ]))
    ])),
    ('nb_clf', MultinomialNB()) # classifier
])

gs_NB2= GridSearchCV(pipeline_NB2, param_grid1, cv=10, n_jobs=2, verbose=1) # same param_grid as NB1
gs_NB2.fit(X_train, y_train)
y_NB2 = gs_NB2.predict(X_test)
accuracy_score(y_test, y_NB2)

Fitting 2 folds for each of 8 candidates, totalling 16 fits


/Users/debora/Envs/nlppython/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=2.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  16 out of  16 | elapsed:  1.3min finished


0.7331251858459709

In [39]:
#have a try with SGD, same features!
pipeline_SGD = Pipeline([
    ('feats', FeatureUnion([
        ('ngram_tfidf', Pipeline([
            ('ngram', CountVectorizer(ngram_range=(1, 4), analyzer='word')),
            ('tfidf', TfidfTransformer()), 
        ])),
        ('ave', AverageWordLengthExtractor())
    ])),
    ('SGD_clf', SGDClassifier())# classifier
])

In [40]:
grid_param_SGD = {'SGD_clf__loss': ['hinge', 'log'],
                  'SGD_clf__penalty': ['none', 'l1', 'l2'],
                  'SGD_clf__max_iter': [50, 100, 500, 1000]}

In [ ]:
gs_SGD = GridSearchCV(pipeline_SGD, grid_param_SGD, cv=10, n_jobs=2, verbose=1)
gs_SGD.fit(X_train, y_train)
##NOTE: it crashed^s at the same point: could not convert string to float: 'ញាំថ្នាំផ្តាសសាយខ្លាំពេកឡើងគេញលែងចង់ចង់ក្រោកហើយ....'
#Maybe some problem with the encoding?

In [ ]:
y_SGD = gs_SGD.predict(X_test)

In [60]:
accuracy_score(y_test, y_SGD)

0.7609277430865299

# Part 2 - MLP

In [62]:
from sklearn.neural_network import MLPClassifier

MLP_CLF = 'MLP_clf'

pipeline_MLP = Pipeline([
    ('features', FeatureUnion([
        # first feature
        ('ngram_tfidf', Pipeline([
            ('ngram', CountVectorizer(ngram_range=(1, 4), analyzer='word')),
            ('tfidf', TfidfTransformer())
        ])),
        # second feature
        ('ave_scaled', Pipeline([
            ('ave', AverageWordLengthExtractor()),
            ('scale', Normalizer())
        ]))
    ])),
    (MLP_CLF, MLPClassifier()) 
])

In [69]:
grid_param_MLP = { MLP_CLF + '__hidden_layer_sizes': [(4, 3), (5, 3)],
                   MLP_CLF + '__activation': ['tanh', 'relu'],
                   MLP_CLF + '__solver': ['sgd', 'adam'],
                   MLP_CLF + '__max_iter': [50],
                   MLP_CLF + '__momentum': [0.9]}

In [ ]:
gs_MLP = GridSearchCV(pipeline_MLP, grid_param_MLP, cv=10, n_jobs=2, verbose=1)
gs_MLP.fit(X_train, y_train)

Fitting 2 folds for each of 8 candidates, totalling 16 fits


/usr/local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=2.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
